## Author: Jesús Salazar Gómez, Costa Rica
## Course: Applied Data Science Capstone, week 4 and 5 (project), Coursera

## LinkedIn: https://www.linkedin.com/in/jesagocr/
## GitHub: https://github.com/jesagocr

In [5]:
# to read the data and do wrangling
from urllib.request import urlopen
from collections import defaultdict


# to work with the api
import json
import requests
import pandas as pd


# to work with the geolocalization data
!pip install geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print('Libraries imported.')

You should consider upgrading via the 'c:\users\jesag\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\jesag\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.
Libraries imported.


In [1]:
# to read the data and do wrangling
from urllib.request import urlopen
from collections import defaultdict

# to work with the api
import json
import requests
import pandas as pd

# to work with the geolocalization data
#!pip install geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!pip install folium
import folium # map rendering library

from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


In [2]:
source = requests.get("https://es.wikipedia.org/wiki/Anexo:Playas_de_Costa_Rica").text
soup = BeautifulSoup(source, 'html.parser')

In [3]:
# primer intento de extraer la info con las playas

playas = []

for grossbeach in soup.find_all('li'):

    texto = str(grossbeach.text)
    #print(texto)

    try:
        if texto[:3] == "Pla" or texto[:3] == "Bal":
            #print('yes')
            playas.append(texto)
        else:
            #no.append(texto)
            pass
    except IndexError:
        pass

In [4]:
def newLine():
    print('********* -*- ********* ')

In [5]:
print(playas)

['Playa Barra del Colorado', 'Playa Bonita', 'Playa Cahuita (Blanca)', 'Playa Chiquita', 'Playa Gandoca', 'Playa Manzanillo', 'Playa Moín', 'Playa Negra (Puerto Viejo)', 'Playa Negra (Cahuita)', 'Playa Pirikiki', 'Playa Portete', 'Playa Puerto Vargas', 'Playas Tortuguero', 'Playa Vizcaya', 'Playa Westfalia', 'Playa Arbolito', 'Playa Avellanas', 'Playa Azul (Jurel)', 'Playa Bahía Junquillal', 'Playa Barco Quebrado', 'Playa Barrigona', 'Playa Bejuco', 'Playa Blanca (Santa Elena)', 'Playa Blanca y Virador', 'Playa Bongo', 'Playa Brasilito', 'Playa Buena Vista', 'Playa Camaronal', 'Playa Carbón', 'Playa Carmen', 'Playa Carrillo', 'Playa Cóncava', 'Playa Conchal', 'Playa Coyote', 'Playa Coyotera', 'Playa Cuajiniquil', 'Playa Culebra', 'Playas del Coco', 'Playa Flamingo', 'Playa Garza', 'Playa Grande', 'Playa Gringo', 'Playa Guiones', 'Playa Hermosa', 'Playa Iguanita', 'Playa Islita y Corozalito', 'Playa Jobo', 'Playa Junquillal', 'Playa La Penca', 'Playa Lagarto', 'Playa Langosta', 'Playa M

In [6]:
#todas los nombres con P o B (Playa o Balneario)

print(len(playas))
newLine()

print(playas[-1])
newLine()

playas = playas[:-1]
print(playas[-1])
newLine()

print(len(playas))

137
********* -*- ********* 
Playas de Costa Rica
********* -*- ********* 
Playa Zancudo
********* -*- ********* 
136


In [15]:
# Segundo intento sin BeatifulSoup para extraer quienes tienen info y quienes no
url = urlopen("https://es.wikipedia.org/wiki/Anexo:Playas_de_Costa_Rica")

In [16]:
content = str(url.read(),encoding='utf-8')
print(type(content))

<class 'str'>


In [12]:
#content = str(url.readline(), encoding='utf-8')
#print(type(content))

<class 'str'>


In [52]:
print(content[8000:10000]) # the info I need has a li tag

ipedia/commons/thumb/8/88/Cahuita.jpg/440px-Cahuita.jpg 2x" data-file-width="1600" data-file-height="1082" /></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/Archivo:Cahuita.jpg" class="internal" title="Aumentar"></a></div>Playa Blanca en Cahuita.</div></div></div>
<ul><li><a href="/wiki/Refugio_Nacional_de_Fauna_Silvestre_Barra_del_Colorado" class="mw-redirect" title="Refugio Nacional de Fauna Silvestre Barra del Colorado">Playa Barra del Colorado</a></li>
<li><a href="/w/index.php?title=Playa_Bonita_(Lim%C3%B3n)&amp;action=edit&amp;redlink=1" class="new" title="Playa Bonita (Limón) (aún no redactado)">Playa Bonita</a></li>
<li><a href="/w/index.php?title=Playa_Cahuita_(Blanca)&amp;action=edit&amp;redlink=1" class="new" title="Playa Cahuita (Blanca) (aún no redactado)">Playa Cahuita (Blanca)</a></li>
<li><a href="/wiki/Playa_Chiquita" title="Playa Chiquita">Playa Chiquita</a></li>
<li><a href="/w/index.php?title=Punta_Cocles&amp;action=edit&amp;redlink=1" class="new

In [60]:
data = content.split('<li><a href="')

In [61]:
print(data[1]) # the list of beaches start at index 1

/wiki/Refugio_Nacional_de_Fauna_Silvestre_Barra_del_Colorado" class="mw-redirect" title="Refugio Nacional de Fauna Silvestre Barra del Colorado">Playa Barra del Colorado</a></li>



In [62]:
### need to understand the data
for d in data[1:12]:
    print(d)

/wiki/Refugio_Nacional_de_Fauna_Silvestre_Barra_del_Colorado" class="mw-redirect" title="Refugio Nacional de Fauna Silvestre Barra del Colorado">Playa Barra del Colorado</a></li>

/w/index.php?title=Playa_Bonita_(Lim%C3%B3n)&amp;action=edit&amp;redlink=1" class="new" title="Playa Bonita (Limón) (aún no redactado)">Playa Bonita</a></li>

/w/index.php?title=Playa_Cahuita_(Blanca)&amp;action=edit&amp;redlink=1" class="new" title="Playa Cahuita (Blanca) (aún no redactado)">Playa Cahuita (Blanca)</a></li>

/wiki/Playa_Chiquita" title="Playa Chiquita">Playa Chiquita</a></li>

/w/index.php?title=Punta_Cocles&amp;action=edit&amp;redlink=1" class="new" title="Punta Cocles (aún no redactado)">Punta Cocles</a></li>

/w/index.php?title=Playa_Gandoca&amp;action=edit&amp;redlink=1" class="new" title="Playa Gandoca (aún no redactado)">Playa Gandoca</a></li>

/w/index.php?title=Playa_Manzanillo_(Lim%C3%B3n)&amp;action=edit&amp;redlink=1" class="new" title="Playa Manzanillo (Limón) (aún no redactado)">

In [53]:
from collections import defaultdict

In [124]:
# better a dict with info about which places have info (I guess because are popular) and which not
# I am going to add more categories to this dict so lets prepare for it

playas2 = defaultdict(dict)

for d in data[1:-1]:
    inicio = (d.find('">')+2)
    final = d.find('</')
    name = d[inicio:final]

    # it seems the most popular beaches are complete but the others are "aún no redactado"
    ini = d.find('aún no')

    if ini != -1:   # if "aún no redactado" = no info of the beach, it is possible is not a popular beach
        #playas2[name] = False
        playas2[name]['Info'] = False

    else:
        #playas2[name] = True
        playas2[name]['Info'] = True

In [125]:
# also this dict have more beaches with punta or puerto that I did not count before
playas2

defaultdict(dict,
            {'Playa Barra del Colorado': {'Info': True},
             'Playa Bonita': {'Info': False},
             'Playa Cahuita (Blanca)': {'Info': False},
             'Playa Chiquita': {'Info': True},
             'Punta Cocles': {'Info': False},
             'Playa Gandoca': {'Info': False},
             'Playa Manzanillo': {'Info': False},
             'Playa Moín': {'Info': False},
             'Playa Negra (Puerto Viejo)': {'Info': False},
             'Playa Negra (Cahuita)': {'Info': True},
             'Playa Pirikiki': {'Info': False},
             'Playa Portete': {'Info': False},
             'Playa Puerto Vargas': {'Info': False},
             'Playas Tortuguero': {'Info': True},
             'Playa Vizcaya': {'Info': False},
             'Playa Westfalia': {'Info': False},
             'Puerto Viejo de Talamanca': {'Info': True},
             'Punta Mona': {'Info': True},
             'Punta Uva': {'Info': False},
             'Playa Arbolito': {'Info

In [136]:
len(playas2.keys())

138

### I new web with more info to add categories
### There are 3 categories of beaches, use it to separete the places

In [28]:
source = requests.get("https://www.govisitcostarica.co.cr/travelInfo/beaches/beaches-list.asp").text

In [29]:
soup2 = BeautifulSoup(source, 'html.parser')

In [30]:
for item in soup2.find_all('<div class="pageListContentRow">'): # I need this but I need more practice with BS, it seems it just 
    print(item.text)                                            # works with tags and not other text
                                                                # better use another approach

In [31]:
#like this:
url2 = urlopen("https://www.govisitcostarica.co.cr/travelInfo/beaches/beaches-list.asp")

In [32]:
contenido = str(url2.read(), encoding='utf-8')

In [33]:
types = contenido.split('<div class="pageListContentRow">') #each list of beaches are listed in a column with the class"pageList..."

In [34]:
soup = BeautifulSoup(types[1], 'html.parser') # we splited the data and select the first column with data (index 1)
for d in soup.find_all('div'):                # this case it only have the categories  
    print(d.text)

Surfing Beaches
Snorkeling Beach


In [37]:
listass = [1,2]
print(listass.index(2))

1


In [95]:
soup = BeautifulSoup(types[2], 'html.parser') # then select the second column or the second pageListContent with our data
count = 0 
cate_data = []
divider = []                                  # 2 index this case, the first div includes / covers all the beaches of 1 category 
                                              # but we can omit it and use the others div that represent just 1 beach
for d in soup.find_all('div'):                
    #print(d.text)                             
    #print(len(d.text))
    cate_data.append(d.text)
    if len(d.text) > 50:
        divider.append(count)
    count = count + 1

In [91]:
print(cate_data[:10])
print(divider)

['Bahía Salinas CahuitaP.N. CahuitaRefugio CamaronalDominicalEl RobleEsterillos EsteEsterillos OesteGarzaJacóP.N. Marino las Baulas MalpaísManuel AntonioMarbellaMatapaloPavonesPlaya AvellanaPlaya Bahia BallenaPlaya BejucoPlaya CaletasPlaya CoyotePlaya Doña AnaPlaya GrandePlaya GuionesPlaya HermosaPlaya JunquillalPlaya LangostaPlaya NegraPlaya NosaraPlaya San MiguelPlaya ZancudoPuerto LimonPuerto SoleyPuerto Viejo de TalamancaQueposP. N. Santa RosaSanta TeresaTamarindo', 'Bahía Salinas ', 'Cahuita', 'P.N. Cahuita', 'Refugio Camaronal', 'Dominical', 'El Roble', 'Esterillos Este', 'Esterillos Oeste', 'Garza']
[0, 39]


In [200]:
surfing_b = cate_data[1:39]
snorkeling_b = cate_data[40:]

In [106]:
surfing_b

['Bahía Salinas ',
 'Cahuita',
 'P.N. Cahuita',
 'Refugio Camaronal',
 'Dominical',
 'El Roble',
 'Esterillos Este',
 'Esterillos Oeste',
 'Garza',
 'Jacó',
 'P.N. Marino las Baulas ',
 'Malpaís',
 'Manuel Antonio',
 'Marbella',
 'Matapalo',
 'Pavones',
 'Playa Avellana',
 'Playa Bahia Ballena',
 'Playa Bejuco',
 'Playa Caletas',
 'Playa Coyote',
 'Playa Doña Ana',
 'Playa Grande',
 'Playa Guiones',
 'Playa Hermosa',
 'Playa Junquillal',
 'Playa Langosta',
 'Playa Negra',
 'Playa Nosara',
 'Playa San Miguel',
 'Playa Zancudo',
 'Puerto Limon',
 'Puerto Soley',
 'Puerto Viejo de Talamanca',
 'Quepos',
 'P. N. Santa Rosa',
 'Santa Teresa',
 'Tamarindo']

In [201]:
snorkeling_b

['Refugio Bahía Junquillal',
 'P.N. Ballena',
 'Cahuita',
 'P.N. Cahuita',
 'Isla del Caño',
 'P. N. Isla del Coco',
 'Drake Bay',
 'Isla Ballena',
 'Manuel Antonio',
 'Manzanillo',
 'Montezuma',
 'Golfo de Papagayo',
 'Playa Carrillo',
 'Playa Chiquita',
 'Playa Conchal',
 'Playa Hermosa',
 'Playa Herradura',
 'Playa Ocotal',
 'Playa Pan de Azucar',
 'Playa Panamá',
 'Playa Potrero',
 'Playa Uvita',
 'Playa Zancudo',
 'Puerto Limon',
 'Puerto Viejo de Talamanca',
 'San Juanillo',
 'Tortuga Abajo',
 'Isla Tortuga']

In [104]:
x = [ 1 for b in cate_data if b in playas2]
print(len(x))

22


In [123]:
x = [ 1 for b in Swimming_b if b in playas2]
print(len(x))

21


In [108]:
soup = BeautifulSoup(types[3], 'html.parser')
for d in soup.find_all('div'):
    print(d.text)

Swimming Beaches



In [113]:
soup = BeautifulSoup(types[4], 'html.parser')
count2 = 0
another_cate = []
#another_diveder = []


for d in soup.find_all('div'):
    #print(d.text)

    if len(d.text) > 50:
        pass
        #another_diveder.append(count2)
    else: 
        another_cate.append(d.text)
        
    count2 = count2 + 1

print(count2)

280


In [159]:
swimming_b = another_cate[:68]
swimming_b

['Refugio Bahía Junquillal',
 'P.N. Ballena',
 'Cabuya',
 'Cahuita',
 'P.N. Cahuita',
 'Isla del Caño',
 'Coronado',
 'Refugio Curú',
 'Drake Bay',
 'El Roble',
 'Garza',
 'Refugio Golfito',
 'Reserva Golfo Dulce',
 'Hacienda Pinilla',
 'Isla Ballena',
 'Islita',
 'Jacó',
 'La Palma',
 'P.N. Marino las Baulas ',
 'Malpaís',
 'Manuel Antonio',
 'P.N. Manuel Antonio',
 'Manzanillo',
 'Marbella',
 'Matapalo',
 'Montezuma',
 'Golfo de Papagayo',
 'Paquera',
 'Parismina',
 'P. N. Piedras Blancas',
 'Playa Bahia Ballena',
 'Playa Bejuco',
 'Playa Brasilito',
 'Playa Carrillo',
 'Playa Cativo',
 'Playa Chiquita',
 'Playa Conchal',
 'Playa Coyote',
 'Playa Flamingo',
 'Playa Grande',
 'Playa Guiones',
 'Playa Hermosa',
 'Playa Herradura',
 'Playa Manzanillo',
 'Playa Naranjo',
 'Playa Nosara',
 'Playa Ocotal',
 'Playa Pan de Azucar',
 'Playa Panamá',
 'Playa Pinuela',
 'Playa Potrero',
 'Playa Real',
 'Playa Tortuga',
 'Playa Uvita',
 'Playa Zancudo',
 'Playas del Coco',
 'Puerto Jiménez',
 'P

In [176]:
b2 = list(playas2.keys())[:-1]
print(b2)
print(type(b2))

['Playa Barra del Colorado', 'Playa Bonita', 'Playa Cahuita (Blanca)', 'Playa Chiquita', 'Punta Cocles', 'Playa Gandoca', 'Playa Manzanillo', 'Playa Moín', 'Playa Negra (Puerto Viejo)', 'Playa Negra (Cahuita)', 'Playa Pirikiki', 'Playa Portete', 'Playa Puerto Vargas', 'Playas Tortuguero', 'Playa Vizcaya', 'Playa Westfalia', 'Puerto Viejo de Talamanca', 'Punta Mona', 'Punta Uva', 'Playa Arbolito', 'Playa Avellanas', 'Playa Azul (Jurel)', 'Playa Bahía Junquillal', 'Playa Barco Quebrado', 'Playa Barrigona', 'Playa Bejuco', 'Playa Blanca (Santa Elena)', 'Playa Blanca y Virador', 'Playa Bongo', 'Playa Brasilito', 'Playa Buena Vista', 'Playa Camaronal', 'Playa Carbón', 'Playa Carmen', 'Playa Carrillo', 'Playa Cóncava', 'Playa Conchal', 'Playa Coyote', 'Playa Coyotera', 'Playa Cuajiniquil', 'Playa Culebra', 'Playas del Coco', 'Playa Flamingo', 'Playa Garza', 'Playa Grande', 'Playa Gringo', 'Playa Guiones', 'Playa Hermosa', 'Playa Iguanita', 'Playa Islita y Corozalito', 'Playa Jobo', 'Playa Ju

In [170]:
total = 0
for b in swimming_b:
    if b in playas2:
        #print(b)
        total += 1
print(total)

21


In [180]:
len(swimming_b)

68

In [186]:
total2 = 0
playa_counter = swimming_b[0]
ind = []


for playa in swimming_b:
    #print(playa)
    for p in b2:
        #print(p)
        #if playa_counter != playa :
        
        if playa in p:
            #print(playa)
            total2 += 1
            ind.append(swimming_b.index(playa))
            
        #print(playa)

    
print(total2)
ind = set(ind)
print(ind)

43
{2, 3, 10, 15, 16, 20, 22, 23, 24, 25, 31, 32, 33, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 50, 51, 52, 53, 54, 55, 57, 61, 63, 64, 65}


In [193]:
beaches_no = [ x for x in swimming_b if swimming_b.index(x) not in ind] 

In [194]:
beaches_no

['Refugio Bahía Junquillal',
 'P.N. Ballena',
 'P.N. Cahuita',
 'Isla del Caño',
 'Coronado',
 'Refugio Curú',
 'Drake Bay',
 'El Roble',
 'Refugio Golfito',
 'Reserva Golfo Dulce',
 'Hacienda Pinilla',
 'Isla Ballena',
 'La Palma',
 'P.N. Marino las Baulas ',
 'Malpaís',
 'P.N. Manuel Antonio',
 'Golfo de Papagayo',
 'Paquera',
 'Parismina',
 'P. N. Piedras Blancas',
 'Playa Bahia Ballena',
 'Playa Cativo',
 'Playa Pan de Azucar',
 'Playa Pinuela',
 'Puerto Jiménez',
 'Punta Banco',
 'Punta Leona',
 'Samara',
 'P. N. Santa Rosa',
 'Tortuga Abajo',
 'Isla Tortuga']

In [190]:
swimming_b.index('Refugio Bahía Junquillal')

0

In [195]:
for playa in swimming_b:
    #print(playa)
    for p in b2:
        #print(p)
        #if playa_counter != playa :
        
        if playa in p:
            #print(playa)
            #total2 += 1
            #ind.append(swimming_b.index(playa))
            playas2[p]['Swimming'] = True 

In [197]:
for playa in surfing_b:
    #print(playa)
    for p in b2:
        #print(p)
        #if playa_counter != playa :
        
        if playa in p:
            #print(playa)
            #total2 += 1
            #ind.append(swimming_b.index(playa))
            playas2[p]['Surfing'] = True

In [202]:
for playa in snorkeling_b:
    #print(playa)
    for p in b2:
        #print(p)
        #if playa_counter != playa :
        
        if playa in p:
            #print(playa)
            #total2 += 1
            #ind.append(swimming_b.index(playa))
            playas2[p]['Snorkeling'] = True

In [203]:
playas2

defaultdict(dict,
            {'Playa Barra del Colorado': {'Info': True},
             'Playa Bonita': {'Info': False},
             'Playa Cahuita (Blanca)': {'Info': False,
              'category': 'swimming',
              'surfing': True,
              'snorkeling': True},
             'Playa Chiquita': {'Info': True,
              'category': 'swimming',
              'snorkeling': True},
             'Punta Cocles': {'Info': False},
             'Playa Gandoca': {'Info': False},
             'Playa Manzanillo': {'Info': False,
              'category': 'swimming',
              'snorkeling': True},
             'Playa Moín': {'Info': False},
             'Playa Negra (Puerto Viejo)': {'Info': False, 'surfing': True},
             'Playa Negra (Cahuita)': {'Info': True,
              'category': 'swimming',
              'surfing': True,
              'snorkeling': True},
             'Playa Pirikiki': {'Info': False},
             'Playa Portete': {'Info': False},
             

In [54]:
de = {}

In [56]:
from collections import defaultdict

In [57]:
de = defaultdict(dict)

In [58]:
de['playa1']['style'] = 'surf'

In [59]:
de

defaultdict(dict, {'playa1': {'style': 'surf'}})

In [60]:
de['playa1']['popular'] = True

In [61]:
de

defaultdict(dict, {'playa1': {'style': 'surf', 'popular': True}})

In [75]:
#to practice let's find the location of the first place in the list

address = 'Playa Zancudo, Costa Rica'

geolocator = Nominatim(user_agent="cr_explorer")
location = geolocator.geocode(address)
print(location)
#latitude = location.latitude
#longitude = location.longitude
#print(type(latitude))
#li.append(latitude)
#print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

Playa Zancudo, Ferrocarril del Sur, Kilometro 7, Florida, Golfito, Cantón Golfito, 60701, Costa Rica


In [71]:
beaches = playas2.keys()
len(beaches)

137

In [75]:
#

In [77]:
lista_beaches = list(beaches)
lista_beaches[:10]

['Playa Barra del Colorado',
 'Playa Bonita',
 'Playa Cahuita (Blanca)',
 'Playa Chiquita',
 'Punta Cocles',
 'Playa Gandoca',
 'Playa Manzanillo',
 'Playa Moín',
 'Playa Negra (Puerto Viejo)',
 'Playa Negra (Cahuita)']

In [78]:
latitudes = []
longitudes = []

for beach in lista_beaches[:70]:
    geolocator = Nominatim(user_agent="cr_explorer_js")
    address = beach+", Costa Rica"
    print(address)
    #print(type(address))

    location = geolocator.geocode(address)

    if location == None:
        latitudes.append(None)
        longitudes.append(None)
        
    else:
        latitudes.append(location.latitude)
        longitudes.append(location.longitude)

Playa Barra del Colorado, Costa Rica
Playa Bonita, Costa Rica
Playa Cahuita (Blanca), Costa Rica
Playa Chiquita, Costa Rica
Punta Cocles, Costa Rica
Playa Gandoca, Costa Rica
Playa Manzanillo, Costa Rica
Playa Moín, Costa Rica
Playa Negra (Puerto Viejo), Costa Rica
Playa Negra (Cahuita), Costa Rica
Playa Pirikiki, Costa Rica
Playa Portete, Costa Rica
Playa Puerto Vargas, Costa Rica
Playas Tortuguero, Costa Rica
Playa Vizcaya, Costa Rica
Playa Westfalia, Costa Rica
Puerto Viejo de Talamanca, Costa Rica
Punta Mona, Costa Rica
Punta Uva, Costa Rica
Playa Arbolito, Costa Rica
Playa Avellanas, Costa Rica
Playa Azul (Jurel), Costa Rica
Playa Bahía Junquillal, Costa Rica
Playa Barco Quebrado, Costa Rica
Playa Barrigona, Costa Rica
Playa Bejuco, Costa Rica
Playa Blanca (Santa Elena), Costa Rica
Playa Blanca y Virador, Costa Rica
Playa Bongo, Costa Rica
Playa Brasilito, Costa Rica
Playa Buena Vista, Costa Rica
Playa Camaronal, Costa Rica
Playa Carbón, Costa Rica
Playa Carmen, Costa Rica
Playa C

In [80]:
latitudes.count(None)

12

In [81]:
longitudes.count(None)

12

In [82]:
for beach in lista_beaches[70:]:
    geolocator = Nominatim(user_agent="cr_explorer_js")
    address = beach+", Costa Rica"
    print(address)
    #print(type(address))

    location = geolocator.geocode(address)

    if location == None:
        latitudes.append(None)
        longitudes.append(None)
        
    else:
        latitudes.append(location.latitude)
        longitudes.append(location.longitude)

Playa Pietra, Costa Rica
Playa Pitahaya, Costa Rica
Playa Pleito y San Juanillo, Costa Rica
Playa Pochote (Papagayo), Costa Rica
Playa Potrero, Costa Rica
Playa Real y Roble, Costa Rica
Playa Rajada, Costa Rica
Playa Sámara, Costa Rica
Playa San Miguel, Costa Rica
Playa Santa Teresa, Costa Rica
Playa Surco de Piedra, Costa Rica
Playa Prieta (Papagayo), Costa Rica
Playa Pochote (Salinas), Costa Rica
Playa Potrero Grande, Costa Rica
Playa Puerto Soley, Costa Rica
Playa Tamarindo, Costa Rica
Playa Ventanas, Costa Rica
Balnerario Puntarenas, Costa Rica
Playa Agujas, Costa Rica
Playa Bahía Drake, Costa Rica
Playa Ballena, Costa Rica
Playa Barú, Costa Rica
Playa Blanca, Costa Rica
Playa Boca Vieja, Costa Rica
Playa Cabo Blanco, Costa Rica
Playa Cabuya, Costa Rica
Playa Cocalito, Costa Rica
Playa Cacao, Costa Rica
Playa Caldera, Costa Rica
Playa Carate, Costa Rica
Playa Curú, Costa Rica
Playa de Arco, Costa Rica
Playa de Doña Ana, Costa Rica
Playa Dominical, Costa Rica
Playa Dominicalito, Cos

In [83]:
print(len(latitudes))
print(latitudes.count(None))
print(longitudes.count(None))

137
24
24
